# Use Model


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%cd /face
!pip3 install -r requirements.txt
!python download_models.py

In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

# CoLab

In [ ]:
opt.name = 'males_model' # or 'femail_model'
model = create_model(opt)
model.eval()

# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

In [ ]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

In [ ]:
# Result
# use_webm = False

!pip3 install webm
webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

# FLASK

In [ ]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken '<ngrok token password>'

In [ ]:
import os
from crypt import methods
from flask_ngrok import run_with_ngrok
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
import pandas as pd

UPLOAD_FOLDER = 'static/uploads/'

app = Flask(__name__)
run_with_ngrok(app) 

app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

In [ ]:
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])

def index():
    return render_template('index.html', title='face GAN')

@app.route('/index', methods=['POST'])

def aging():
    age = request.form['age']
    target_age = request.form['target_age']
    gender = request.form['gender']
    image = request.files['image']
    path = os.path.join(app.config["UPLOAD_FOLDER"], image.filename)
    image.save(path)
    
    if gender == "male":
        input = pd.DataFrame({
        'age' : [int(age)],
        'target_age' :[int(target_age)]
        })

        opt.name = 'males_model'
        model = create_model(opt)
        model.eval()

    elif gender == "female":
        input = pd.DataFrame({
        'age' : [int(age)],
        'target_age' :[int(target_age)]
        })

        opt.name = 'females_model'
        model = create_model(opt)
        model.eval()  

    name = f'./static/uploads/{image.filename}'
    data = dataset.dataset.get_item_from_path(name)
    visuals = model.inference(data)

    # Model running (images)
    os.makedirs(f'results/{image.filename}', exist_ok=True)
    out_pathi = f'./results/{image.filename}'   

    visualizer.save_images_deploy(visuals, out_pathi)
    
    # Model running (video)
    os.makedirs('results', exist_ok=True)
    out_pathv = os.path.join('results', os.path.splitext(name)[0].replace(' ', '_') + '.webm')
    visualizer.make_video(visuals, out_pathv)

    return render_template('output.html', filename=image.filename) #Output = ModelOutput)

if __name__ == "__main__":
    app.run()

In [ ]:
@inproceedings{orel2020lifespan,
  title={Lifespan Age Transformation Synthesis},
  author={Or-El, Roy
          and Sengupta, Soumyadip
          and Fried, Ohad
          and Shechtman, Eli
          and Kemelmacher-Shlizerman, Ira},
  booktitle={Proceedings of the European Conference on Computer Vision (ECCV)},
  year={2020}
}